In [3258]:
#//author : Pauline Lelandais in the context of implementing theoretical results from my undergraduate research project
import numpy as np       #library imports
import random as rn
import math

We try to model the modal ray arrival pattern of underwater waves propagating from a sound source. It is assumed we know its source excitation function and the frequency of the signal.
We aim at first producing reference data to compile a library, then we simulate experimental data and try to match it to one of the known profile. This aims at determining the average speed of sound in the environmnet where the experiment took place. As this value changes with temperature, this would be useful for example in monitoring the effect of climate change under water.

In [3259]:
def initialise_clist (cn): #initialise sound speed values
    clist =[0 for i in range(cn)]
    for i in range (cn):
        clist[i]=1450+i*20 #the speed of sound ranges from 1450m/s to 1550m/s in natural environments
    return clist

In [3260]:
def initialise_tlist (nt): # number of data points in t dimension
    tlist =[float(0.05*x+100) for x in range(nt)] #we dont want to start at 0 to avoid division be zero error messages
    return tlist

In [3261]:
def knbr (c,w):
    #Wavenumber for specified c (sound speed profile)
    #and frequency omega
    k = w/c 
    return k

In [3262]:
def _lambda(k, D) :
    #finds value for which the depth problem solves boundary conditions
    L=math.sqrt(k*k-(math.pi*math.pi)/(4*D*D)) #L stands for lambda
    L=abs(L) #want the positive root
    return L

In [3263]:
def find_freq(nbr):
    w_0=2*math.pi*115 #fundamental frequency
    A=80.0
    w=w_0+int((nbr+1)/2)*A*(-1)**(nbr+1)
    return w

In [3264]:
def dtwave_eq (x, t , c) : #find value time derivative at this x=(r,z) 
    r=x[0]
    z=x[1]
    D=750 #prescribed depth of water bottom interface
    pi=math.pi#makes me life easier
    size_quad=1 #2k+1 size of the frequency sample
    S=0 #initialising sum
    for i in range(2*size_quad+1):
        w=find_freq(i) #fundamental frequency
        k=knbr(c, w) #find wave number in specified environment for specified frequency
        L=_lambda(k, D)
        M = -pi/4 - r*L
        cM=math.cos(M) 
        sM=math.sin(M)
        y=-1*w*math.sqrt(2/(pi*r*L))*(cM+sM*1j)*math.sin(pi*z/(2*D))*math.cos(t*w)
        y=y.real
        S+=y
    #it was tidious to write but wait until I explain how long it took me to derive
    return S

In [3265]:
def computedt (c, tlist, x):
    nt=len(tlist)
    fprime =[0 for i in range(nt)] #initialise
    for i in range (nt) : #for each point in time
        fprime[i]=dtwave_eq (x, tlist[i], c)#computes the value of the time derivative
    return fprime

In [3266]:
def evaluate (x): #evaluates the sign of the complex number
    if x >0:
        sign =1
    else :
        sign=-1
    return sign

In [3267]:
def lookforpeak (dtlist, tlist): #for this location in space look for extrema wrt to time
    peaklist =[] #initialise
    nt=len(tlist)
    for i in range (nt-1): #for each point in time
        tell=evaluate(dtlist[i]) #evaluate sign at first t-value
        sign = evaluate (dtlist[i+1]) #find sign next point
        if tell == -1*sign : #derivative switch orientation
            nt= (i+i+1)/2
            peaklist.append(nt)
    return peaklist

In [3268]:
def find_peaks (c, tlist, x) :
    dtlist=computedt(c, tlist, x) #compute values of time derivative at all points in time and position
    peaklist = lookforpeak(dtlist, tlist)
    return peaklist

In [3269]:
def compute_library (clist, tlist, nt, x):#for each sound speed (environment specific)
    cpeaks =[]#initialise list of peaks for each sound speed
    cn =len(clist)
    for i in range (cn):
        c=clist[i] 
        peaklist=find_peaks (c, tlist, x)#compute modal time arrival
        cpeaks.append(peaklist) #append
    return cpeaks

In [3270]:
def random_pick(clist, cpeaks) :
    cn=len(clist)#makes my life easier
    k=rn.randint(0,cn-1) #pick a number between 0 and 20
    l=len(cpeaks[k]) # look how many peaks for that value of c
    if l == 0: # if its empty
        while l==0: #enter a while loop
            k=rn.randint(0,cn-1) #we dont want it so choose another one
            l=len(cpeaks[k]) #hopefully no infinite loop
    return k

In [3271]:
def findscore (ref, l, data, nd):
    score=0
    if l==0: #if the list in empty #fail safe
        score=+5000 #not a match
    else :
        score=0 #do nothing #inefficient
    for i in range(nd): #compare each arrival time
        element=data[i]
        for j in range(l):
            if ref[j] == element :
                score+=1
            else :
                score=score #do nothing #inneficient
    score=nd-score#compute how close
    return score

In [3272]:
def matchfield (library, data):
    nd=len(data)#makes my life easier  
    nl=len(library)
    scores=np.array([]) #initialising
    for i in range (nl):
        score = findscore (library[i],len(library[i]), data, len(data) )
        scores=np.append(scores, score)#record score  
    match=scores.argmin()#find closest match
    return match

In [3273]:
def run_process (sample, clist, cpeaks, tlist, x):
    S=0.0
    for i in range(sample):
        #now we simulate experimental data by picking a c value at random and modifying by epsilon
        k=random_pick(clist, cpeaks)
        modk=clist[k]+eps
        _data = find_peaks (modk, tlist, x)#we simulate what we would receive from a sonar 
        #in experimental conditions
        c_data=matchfield(cpeaks, _data)#now we try to match our experimental data 
        #with one of the reference environment that we just computed
        y=-1.0 #initialising
        if c_data == k : # if the c value in environment we experimented in
            #is the c value the score function estimates to be the closest match to the experimental environment
            y=1.0 #we win 
        else :
            y=0.0 #else we lose 
        S+=y
    performance = float(S/sample*100) #average score
    print performance

In [3274]:
cn=5 # size of reference library
eps = float(0.002) # experimental modification
nt=100 # number a time & local variables are considered
x= [15000, 400] #position of hydrophone
sample=100 #number of times we repeat the experiment

clist= initialise_clist(cn) #initialise sound speed values
tlist = initialise_tlist(nt) #initialise time array
cpeaks = compute_library (clist, tlist, nt, x)#computes the reference sound speed profiles

run_process(sample, clist, cpeaks, tlist, x)
#models experimental data and runs match field process + perfomance analysis

81.0
